<a href="https://colab.research.google.com/github/amien1410/amien-scrapers/blob/main/Tribun_News_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import json
import requests
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
url = "https://cse.google.com/cse/element/v1?rsz=filtered_cse&num=10&hl=en&source=gcsc&start=0&cselibv=6467658b9628de43&cx=4593c3f3750fa44b5&q=khalid+basalamah&safe=active&cse_tok=AEXjvhLV-IG2cfxG6BXvAU7fPlNh%3A1758288767458&lr=&cr=&gl=&filter=0&sort=date&as_oq=&as_sitesearch=&exp=cc%2Capo&callback=google.search.cse.api5313&rurl=https%3A%2F%2Fwww.tribunnews.com%2Fsearch%3Fq%3Dkhalid%2Bbasalamah%26cx%3Dpartner-pub-f6e618dd8369c4970%26cof%3DFORID%253A10%26ie%3DUTF-8%26siteurl%3Dtribunnews.com"
headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7', 'accept-encoding': 'gzip, deflate, br, zstd', 'accept-language': 'en-US,en;q=0.9,id;q=0.8,fa;q=0.7,ar;q=0.6,ms;q=0.5,ja;q=0.4,es;q=0.3', 'cookie': 'ukid=628cb0f6cb42d86e1e0a7217cc5c0e8c; 3bun_session=7ov4m4i3v4mt03hhbksuub9vi5pn7tpc; vignette_cookies_tribun=1; g_state={"i_p":1758295727682,"i_l":1}; FCNEC=%5B%5B%22AKsRol_pIImZGqpS-lqTTovNRJTh4rP2tcxHZfRMF2YzWIVpLprpcacPdZW3vq55zP17hQ1Z03Uy4yks6eG0Pd1fWGktPrAFWuhq4oMfmG9ZJ-4j51aZkoi1Dvt-cUgnR6IaAdVcKO37gIlTawggKKA2UOyyxWSERQ%3D%3D%22%5D%5D', 'priority': 'u=0, i', 'sec-ch-ua': '"Chromium";v="140", "Not=A?Brand";v="24", "Microsoft Edge";v="140"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': '"Windows"', 'sec-fetch-dest': 'document', 'sec-fetch-mode': 'navigate', 'sec-fetch-site': 'none', 'sec-fetch-user': '?1', 'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.0.0'}
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')
script_text = soup.text.strip()

# Remove wrapper
cleaned = re.sub(r"^/\*.*\*/\s*", "", script_text)           # remove comment
cleaned = re.sub(r"^google\.search\.cse\.api5313\(", "", cleaned) # remove function call
cleaned = re.sub(r"\);$", "", cleaned)               # remove trailing );

data = json.loads(cleaned)
data['results']

In [ ]:
from datetime import datetime, timedelta
import re

def convert_relative_time(time_str):
    """
    Convert relative time strings (e.g., '2 hours ago', '3 days ago')
    into an absolute datetime (YYYY-MM-DD HH:MM:SS).
    """
    now = datetime.now()

    # Match pattern like "2 hours ago" or "3 days ago"
    match = re.match(r"(\d+)\s+(second|minute|hour|day|week|month|year)s?\s+ago", time_str.strip())
    if not match:
        return None

    value, unit = int(match.group(1)), match.group(2)

    if unit == "second":
        dt = now - timedelta(seconds=value)
    elif unit == "minute":
        dt = now - timedelta(minutes=value)
    elif unit == "hour":
        dt = now - timedelta(hours=value)
    elif unit == "day":
        dt = now - timedelta(days=value)
    elif unit == "week":
        dt = now - timedelta(weeks=value)
    elif unit == "month":
        # Approximate 1 month = 30 days
        dt = now - timedelta(days=30*value)
    elif unit == "year":
        # Approximate 1 year = 365 days
        dt = now - timedelta(days=365*value)
    else:
        return None

    return dt.strftime("%Y-%m-%d %H:%M:%S")

# Example usage
print(convert_relative_time("2 hours ago"))
print(convert_relative_time("3 days ago"))
print(convert_relative_time("5 weeks ago"))
print(convert_relative_time("1 year ago"))

2025-09-19 11:56:19
2025-09-16 13:56:19
2025-08-15 13:56:19
2024-09-19 13:56:19


In [ ]:
news = []
page = 1

while True:
    url2 = f"https://search.kompas.com/search?q=khalid+basalamah&site_id=all&start_date=2025-08-01&end_date=2025-09-19&sort=latest&page={page}"
    res = requests.get(url2)
    soup = BeautifulSoup(res.text, 'html.parser')

    # Find all div with class "articleItem"
    articles = soup.find_all("div", class_="articleItem")

    # Stop if less than 20 articles found
    if len(articles) < 20:
        break

    for article in articles:
        # Extract date string
        date_str = article.find("div", class_="articlePost-date").get_text(strip=True)

        # Convert Indonesian months → English months
        month_map = {
            "Januari": "January", "Februari": "February", "Maret": "March",
            "April": "April", "Mei": "May", "Juni": "June",
            "Juli": "July", "Agustus": "August", "September": "September",
            "Oktober": "October", "November": "November", "Desember": "December"
        }
        for indo, eng in month_map.items():
            if indo in date_str:
                date_str = date_str.replace(indo, eng)

        # Parse into datetime, then format as YYYY-MM-DD
        date_formatted = datetime.strptime(date_str, "%d %B %Y").strftime("%Y-%m-%d")

        news.append({
            "url": article.find("a", class_="article-link")["href"],
            "title": article.find("h2", class_="articleTitle").get_text(strip=True),
            "image": article.find("img")["src"],
            "image_alt": article.find("img").get("alt"),
            "category": article.find("div", class_="articlePost-subtitle").get_text(strip=True),
            "date": date_formatted,
            "lead": article.find("div", class_="articleLead").get_text(strip=True),
        })

    page += 1  # go to next page

# Convert to DataFrame
df = pd.DataFrame(news)
df

,url,title,image,image_alt,category,date,lead
0,https://www.kompas.com/tren/read/2025/09/19/20...,"Soal Korupsi Haji, Ini Alasan Khalid Basalamah...",https://asset.kompas.com/crops/_Ao1vmKtBUX1jN5...,"Soal Korupsi Haji, Ini Alasan Khalid Basalamah...",Tren,2025-09-19,KPK mengungkap alasan Khalid Basalamah mengemb...
1,https://www.kompas.com/kalimantan-timur/read/2...,"Ustaz Basalamah Diperas Oknum Kemenag, Ini Kat...",https://asset.kompas.com/crops/_ASgaXaTTvq6wc4...,"Ustaz Basalamah Diperas Oknum Kemenag, Ini Kat...",Kalimantan Timur,2025-09-19,"Dirjen PHU Kementerian Agama, Hilman Latief, b..."
2,https://www.kompas.com/jawa-tengah/read/2025/0...,KPK Ungkap Oknum Kemenag Peras Ustaz Basalamah...,https://asset.kompas.com/crops/AWDspyau27Urbu3...,KPK Ungkap Oknum Kemenag Peras Ustaz Basalamah...,Jawa Tengah,2025-09-19,KPK mengungkap ada oknum Kemenag yang mengemba...
3,https://video.kompas.com/watch/1877048/peras-k...,"Peras Khalid Basalamah, Oknum Kemenag Minta 2....",https://assets-studiohub.kompas.com/video2019/...,"Peras Khalid Basalamah, Oknum Kemenag Minta 2....",Video,2025-09-19,Oknum Kemenag disebut meminta 2.400 - 7.000 US...
4,https://nasional.kompas.com/read/2025/09/19/16...,Dirjen PHU Buka Suara soal Khalid Basalamah Di...,https://asset.kompas.com/crops/_ASgaXaTTvq6wc4...,Dirjen PHU Buka Suara soal Khalid Basalamah Di...,News,2025-09-19,Dirjen PHU Kemenag Hilman Latief buka suara so...
5,https://nasional.kompas.com/read/2025/09/19/14...,"Pansus Haji DPR Buat Oknum Kemenag Ketakutan, ...",https://asset.kompas.com/crops/SmtJXZBMsEUfPVc...,"Pansus Haji DPR Buat Oknum Kemenag Ketakutan, ...",News,2025-09-19,KPK menyebut oknum Kemenag inilah yang menawar...
6,https://nasional.kompas.com/read/2025/09/19/14...,KPK Sebut Khalid Basalamah Pindah ke Haji Khus...,https://asset.kompas.com/crops/AWDspyau27Urbu3...,KPK Sebut Khalid Basalamah Pindah ke Haji Khus...,News,2025-09-19,KPK mengungkap alasan Khalid Basalamah pindah ...
7,https://www.kompas.com/banten/read/2025/09/19/...,KPK Bongkar Dugaan Jemaah Haji Tergiur Maktab ...,https://asset.kompas.com/crops/3gVQeKnM3yQ3V6a...,KPK Bongkar Dugaan Jemaah Haji Tergiur Maktab ...,Banten,2025-09-19,KPK curiga iming-iming maktab VIP haji bukan c...
8,https://www.kompas.com/lampung/read/2025/09/19...,Skandal Kuota Haji Rp 1 Triliun: KPK Beberkan ...,https://asset.kompas.com/crops/3gVQeKnM3yQ3V6a...,Skandal Kuota Haji Rp 1 Triliun: KPK Beberkan ...,Lampung,2025-09-19,KPK mengungkap alasan Khalid Basalamah mengemb...
9,https://nasional.kompas.com/read/2025/09/19/10...,"Peras Khalid Basalamah, Oknum Kemenag Minta 2....",https://asset.kompas.com/crops/zNOOy0urTONeVI4...,"Peras Khalid Basalamah, Oknum Kemenag Minta 2....",News,2025-09-19,KPK mengungkap dugaan korupsi dalam penentuan ...


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.parse import quote_plus

queries = ["khalid basalamah", "yaqut", "korupsi kuota haji", "ibnu mas'ud"]
news = []

month_map = {
    "Januari": "January", "Februari": "February", "Maret": "March",
    "April": "April", "Mei": "May", "Juni": "June",
    "Juli": "July", "Agustus": "August", "September": "September",
    "Oktober": "October", "November": "November", "Desember": "December"
}

for query in queries:
    page = 1
    while True:
        # encode query for URL
        query_encoded = quote_plus(query)

        url = f"https://search.kompas.com/search?q={query_encoded}&site_id=all&start_date=2025-08-01&end_date=2025-10-04&sort=latest&page={page}"
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')

        # Find all div with class "articleItem"
        articles = soup.find_all("div", class_="articleItem")

        # Stop if less than 20 articles found
        if len(articles) < 20:
            break

        for article in articles:
            # Extract date string
            date_str = article.find("div", class_="articlePost-date").get_text(strip=True)

            # Convert Indonesian months → English months
            for indo, eng in month_map.items():
                if indo in date_str:
                    date_str = date_str.replace(indo, eng)

            # Parse into datetime, then format as YYYY-MM-DD
            date_formatted = datetime.strptime(date_str, "%d %B %Y").strftime("%Y-%m-%d")

            news.append({
                "query": query,
                "url": article.find("a", class_="article-link")["href"],
                "title": article.find("h2", class_="articleTitle").get_text(strip=True),
                "image": article.find("img")["src"],
                "image_alt": article.find("img").get("alt"),
                "category": article.find("div", class_="articlePost-subtitle").get_text(strip=True),
                "date": date_formatted,
                "lead": article.find("div", class_="articleLead").get_text(strip=True),
            })

        page += 1  # next page

# Convert to DataFrame
df = pd.DataFrame(news)
df


,query,url,title,image,image_alt,category,date,lead
0,khalid basalamah,https://nasional.kompas.com/read/2025/09/25/22...,KPK Sebut Khalid Basalamah Paling Tahu Oknum K...,https://asset.kompas.com/crops/AWDspyau27Urbu3...,KPK Sebut Khalid Basalamah Paling Tahu Oknum K...,News,2025-09-25,KPK menyatakan Ustaz Khalid Basalamah tahu okn...
1,khalid basalamah,https://video.kompas.com/watch/1878469/kpk-seb...,KPK Sebut Ustaz Khalid Basalamah Tahu Oknum Ka...,https://assets-studiohub.kompas.com/video2019/...,KPK Sebut Ustaz Khalid Basalamah Tahu Oknum Ka...,Video,2025-09-25,"KPK menyebut, Ustaz Khalid Basalamah diduga me..."
2,khalid basalamah,https://www.kompas.com/kalimantan-timur/read/2...,Kenapa Uang Khalid Basalamah yang Disita KPK T...,https://asset.kompas.com/crops/AWDspyau27Urbu3...,Kenapa Uang Khalid Basalamah yang Disita KPK T...,Kalimantan Timur,2025-09-24,KPK jelaskan alasan uang yang dikembalikan Kha...
3,khalid basalamah,https://nasional.kompas.com/read/2025/09/23/15...,KPK Jelaskan Alasan Uang yang Disita dari Khal...,https://asset.kompas.com/crops/p5ZpxsHVpxdrcDC...,KPK Jelaskan Alasan Uang yang Disita dari Khal...,News,2025-09-23,KPK menjelaskan alasan uang yang disita dari U...
4,khalid basalamah,https://nasional.kompas.com/read/2025/09/22/17...,KPK Tak Ungkap Jumlah Uang yang Diserahkan Kha...,https://asset.kompas.com/crops/_Ao1vmKtBUX1jN5...,KPK Tak Ungkap Jumlah Uang yang Diserahkan Kha...,News,2025-09-22,KPK memastikan Ustaz Khalid Basalamah mengemba...
...,...,...,...,...,...,...,...,...
1455,korupsi kuota haji,https://regional.kompas.com/read/2025/08/03/18...,Kejati Kembali Sita Harta Tersangka Korupsi Pe...,https://asset.kompas.com/crops/X1zLdUdiA08cD3a...,Kejati Kembali Sita Harta Tersangka Korupsi Pe...,Regional,2025-08-03,Kejati Bengkulu kembali melakukan penggeledaha...
1456,korupsi kuota haji,https://www.kompas.com/jawa-timur/read/2025/08...,Cara Daftar Upacara 17 Agustus 2025 di Istana ...,https://asset.kompas.com/crops/oCDpa2UIf3mlrZs...,Cara Daftar Upacara 17 Agustus 2025 di Istana ...,Jawa Timur,2025-08-03,Presiden Prabowo mengundang sebanyak 8.000 ora...
1457,korupsi kuota haji,https://money.kompas.com/read/2025/08/03/14104...,"10 Saham Paling Cuan Sepekan Terakhir, Ada Emi...",https://asset.kompas.com/crops/sj6wWAk64HTmN45...,"10 Saham Paling Cuan Sepekan Terakhir, Ada Emi...",Money,2025-08-03,Daftar saham paling cuan pada pekan terakhir J...
1458,korupsi kuota haji,https://www.kompas.com/properti/read/2025/08/0...,"Kuota KPR Subsidi Naik Drastis, BTN Siapkan 22...",https://asset.kompas.com/crops/T64Xq9ym1mRGKMw...,"Kuota KPR Subsidi Naik Drastis, BTN Siapkan 22...",Properti,2025-08-02,PT Bank Tabungan Negara (Persero) Tbk (BTN) me...
